# Spanish language records deduplication

#### Objectives
- Read in data
- Apply cleaning rules
- Apply phonetics encoding
- Construct identifiers
- Evaluate performance

In [2]:
# standard libraries
import time
import re # package to perform regular expressions
import sys
import os
import numpy as np
import pandas as pd
import datetime
import pyarrow # Fast reading of parquets

# letters of the alphabet libraries
import string
from string import ascii_lowercase, ascii_uppercase 
from random import choice
from functools import reduce
import xml.etree.ElementTree as et #xml

#phonetics
import phonetics

# vizualization libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly

# date formatter
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter


# modeling libraries
import sklearn
from sklearn import *
import statsmodels.api as sm

# Import joblib for data persistance
import joblib
import pickle

# check versions
print(f'Pandas version: {pd.__version__}')
print(f'Numpy version: {np.__version__}')
print(f'Joblib version: {joblib.__version__}')
print(f'Seaborn version: {sns.__version__}')
print(f'Plotly version: {plotly.__version__}')
print(f'PyArrow version: {pyarrow.__version__}')

Pandas version: 1.2.1
Numpy version: 1.19.4
Joblib version: 0.17.0
Seaborn version: 0.10.0
Plotly version: 3.9.0
PyArrow version: 3.0.0


In [3]:
# Helper functions
def catvardistribution(data,var,x,y,rot):
    print(data[var].value_counts(dropna=False))
    fig, ax = plt.subplots(figsize=(x,y)) # Set figure size
    sns.set(style="darkgrid")
    ax = sns.countplot(x=data[var], data=data, palette='Reds')
    plt.xticks(rotation=rot)
    plt.show()

def boxplot(data,var):
    print(data[var].describe())
    sns.set(style="whitegrid")
    ax = sns.boxplot(x=data[var])
    plt.show()

### Read in data

In [4]:
# set parameters
date = datetime.datetime.now().date()
directory_input = "C:/..." #set input directory
directory_output = "C:/..." #set output directory

In [5]:
# read parquet to pandas dataframe
d = pd.read_parquet(f'{directory_input}/data.parquet.gzip')  

In [6]:
# preserve original dataframe copy
d_original = d.copy()

#### Inspect the data

In [7]:
d.shape

In [8]:
d.info()

In [9]:
d.tail(2)

In [10]:
d.head(2)

In [11]:
d.iloc[0]

### Data cleaning

In [12]:
def clean_names(df, column):
    df[column] = df[column].str.replace(r'[\s]+','') # remove spaces
    df[column] = df[column].str.replace(r'[\d]+','') # remove numerical digits
    df[column] = df[column].str.replace('Á','A') # replace A with accent mark
    df[column] = df[column].str.replace('É','E') # replace E with accent mark
    df[column] = df[column].str.replace('Í','I') # replace I with accent mark
    df[column] = df[column].str.replace('Ó','O') # replace O with accent mark
    df[column] = df[column].str.replace('Ú','U') # replace U with accent mark
    df[column] = df[column].str.replace('Ì','I') # replace I with reverse accent mark
    df[column] = df[column].str.replace('Ñ','N') # replace Ñ with N
    df[column] = df[column].str.replace('[^A-Za-z]+', '') # remove non-Latin letters
    df[column] = df[column].str.lower() #lower case names
    df[column] = np.where(df[column].isnull(),'xxx',df[column]) # replace nulls with `xxx` 
    df[column] = np.where(df[column]=="",'xxx',df[column]) # replace nulls with `xxx`
    return df

#### Export excel file with missing or placeholder `GENDER`, `LAST_NAME` or `FIRST_NAME`. These records cannot be deduplicated

In [13]:
noidentifier = d[(d['GENDER']=='Not Defined') | (d['FIRST_NAME'].str[0:6]=='XXXXXX') | (d['LAST_NAME'].str[0:6]=='XXXXXX')]
print(len(noidentifier))

In [14]:
noidentifier.head()

In [15]:
print(f'Percentage of data set that are dummies {len(noidentifier)/len(d_original)*100}%')

In [16]:
# drop superfluous columns
noidentifier = noidentifier.drop(list(noidentifier.columns[75:99]), axis=1)

In [17]:
# write dummy records to parquet
start = time.time()
print('Writing pandas dataframe to parquet format..')
noidentifier.to_parquet(f'{directory_output}/noidentifier.parquet.gzip', compression='gzip')
print(f"... completed job in {time.time() - start} seconds")

In [18]:
# drop these not-identifiable records from the data for deduplication
d = d.drop(noidentifier.index)

In [19]:
print(len(d))

In [20]:
# reset index
d = d.reset_index(drop=True)

### Exploratory Data Analysis and Feature Engineering

In [21]:
catvardistribution(d,'GENDER',x=5,y=5,rot=0)

In [22]:
# Assert that there are zero records with Not Defined Gender
len(d[d.GENDER=='Not Defined']) ==0

In [23]:
#inspect concentration of birthdates
d['birthmonthday'].value_counts(dropna=False)/len(d)

In [24]:
# create back up of original whole name columns
for col in d[['FIRST_NAME','MIDDLE_NAME','LAST_NAME']]:
    d[f"{col}_backup"] = d[col]

In [25]:
# create list of names variables

first_names = list()
for col in d.columns:
    if col.startswith('FIRST_NAME_'):
        first_names.append(col)

middle_names = list()
for col in d.columns:
    if col.startswith('MIDDLE_NAME_'):
        middle_names.append(col)

last_names = list()
for col in d.columns:
    if col.startswith('LAST_NAME_'):
        last_names.append(col)

In [26]:
names = [first_names + middle_names + last_names]
names_decomposed = names[0].copy()
print(names_decomposed)

In [27]:
# for clean names decomposition, remove name backups
names_backup = ['FIRST_NAME_backup','MIDDLE_NAME_backup','LAST_NAME_backup']
for x in names_backup:
    names_decomposed.remove(x)
print(names_decomposed)

#### Apply data cleaning rules

In [28]:
# clean names
for name in d[names[0]]:
    d = clean_names(d, name)

In [29]:
for name in names[0]:
    print(name)
    print(d[name].value_counts(dropna=False).head())  

In [30]:
# Generate columns for each of the first 3 letters of the each of the names

# generate lists for each letter variable
letter1 = d[names[0]].columns.astype(str) + '_letter1'
letter2 = d[names[0]].columns.astype(str) + '_letter2'
letter3 = d[names[0]].columns.astype(str) + '_letter3'

letter1 = letter1.tolist()
letter2 = letter2.tolist()
letter3 = letter3.tolist()

In [31]:
# create variable for first letter
for letter,name in zip(letter1, names[0]):
    d[letter] = d[name].str[0]

# create variable for second letter
for letter,name in zip(letter2, names[0]):
    d[letter] = d[name].str[1]

# create variable for third letter
for letter,name in zip(letter3, names[0]):
    d[letter] = d[name].str[2]


In [32]:
d.head()

### Phonetics

In [33]:
# generate lists for each phonetics encoding type
soundex = d[names[0]].columns.astype(str) + '_soundex'
metaphone = d[names[0]].columns.astype(str) + '_metaphone'
dmetaphone = d[names[0]].columns.astype(str) + '_dmetaphone'

soundex = soundex.tolist()
metaphone = metaphone.tolist()
dmetaphone = dmetaphone.tolist()

In [34]:
# create soundex features
for phone,name in zip(soundex, names[0]):
    d[phone] = d[name].apply(lambda x: phonetics.soundex(x))      
        
# create metaphone features
for phone,name in zip(metaphone, names[0]):
    d[phone] = d[name].apply(lambda x: phonetics.metaphone(x)) 

# create double metaphone features
for phone,name in zip(dmetaphone, names[0]):
    d[phone] = d[name].apply(lambda x: phonetics.dmetaphone(x)) 

In [35]:
# check results
for name in soundex:
    print(name)
    print(d[name].value_counts(dropna=False).head())
    
for name in metaphone:
    print(name)
    print(d[name].value_counts(dropna=False).head())
    
for name in dmetaphone:
    print(name)
    print(d[name].value_counts(dropna=False).head())

In [36]:
# for partial names decomposition, remove name backup letters
letter1_backup = ['FIRST_NAME_backup_letter1','MIDDLE_NAME_backup_letter1','LAST_NAME_backup_letter1']
letter2_backup = ['FIRST_NAME_backup_letter2','MIDDLE_NAME_backup_letter2','LAST_NAME_backup_letter2']
letter3_backup = ['FIRST_NAME_backup_letter3','MIDDLE_NAME_backup_letter3','LAST_NAME_backup_letter3']

for x in letter1_backup:
    letter1.remove(x)

for x in letter2_backup:
    letter2.remove(x)

for x in letter3_backup:
    letter3.remove(x)
    
print(letter1)
print(letter2)
print(letter3)

In [37]:
# for phonetics decomposition, remove name backup letters
soundex_backup = ['FIRST_NAME_backup_soundex','MIDDLE_NAME_backup_soundex','LAST_NAME_backup_soundex']
metaphone_backup = ['FIRST_NAME_backup_metaphone','MIDDLE_NAME_backup_metaphone','LAST_NAME_backup_metaphone']
dmetaphone_backup = ['FIRST_NAME_backup_dmetaphone','MIDDLE_NAME_backup_dmetaphone','LAST_NAME_backup_dmetaphone']

for x in soundex_backup:
    soundex.remove(x)

for x in metaphone_backup:
    metaphone.remove(x)

for x in dmetaphone_backup:
    dmetaphone.remove(x)
    
print(soundex)
print(metaphone)
print(dmetaphone)

In [38]:
# create dictionary of engineered features
feature_dict = {'names':names[0], 'letter1': letter1, 'letter2': letter2, 'letter3': letter3, 
                'soundex': soundex, 'metaphone': metaphone, 'dmetaphone': dmetaphone, 'names_decomposed': names_decomposed}

### Construct identifiers for deduplication (letter-based, whole name, soundex, and metaphone-based identifiers)

In [40]:
# identifier 1 (whole names - raw without cleaning + BIRTHDATE)
d['raw_wholename_id'] = d['FIRST_NAME'] + d['LAST_NAME'] + d['GENDER'] + d['birthdate'] 

# identifier 2 (whole names - with cleaning + BIRTHDATE)
d['clean_wholename_id'] = d['FIRST_NAME_backup'] + d['MIDDLE_NAME_backup']+ d['LAST_NAME_backup'] \
                        + d['GENDER'] + d['birthdate']

# identifier 2b (whole names - with cleaning + MOBILE_NUMBER)
d['clean_wholename_mobile_id'] = d['FIRST_NAME_backup'] + d['MIDDLE_NAME_backup']+ d['LAST_NAME_backup'] \
                        + d['GENDER'] + d['MOBILE_NUMBER']

# identifier 2c (whole names - with cleaning + BIRTHYEAR + BIRTHMONTH)
d['clean_wholename_birthyearmonth_id'] = d['FIRST_NAME_backup'] + d['MIDDLE_NAME_backup']+ d['LAST_NAME_backup'] \
                        + d['GENDER'] + d['birthyear_str'] + d['birthmonth_str']

# identifier 2d (whole names - with cleaning + BIRTHYEAR + BIRTHDAY)
d['clean_wholename_birthyearday_id'] = d['FIRST_NAME_backup'] + d['MIDDLE_NAME_backup']+ d['LAST_NAME_backup'] \
                        + d['GENDER'] + d['birthyear_str'] + d['birthday_str']

# identifier 2e (whole names - with cleaning + BIRTHMONTH + BIRTHDAY)
d['clean_wholename_birthmonthday_id'] = d['FIRST_NAME_backup'] + d['MIDDLE_NAME_backup']+ d['LAST_NAME_backup'] \
                        + d['GENDER'] + d['birthmonth_str'] + d['birthday_str']

# identifier 3 (whole decomposed names - with cleaning)
d['clean_wholename_decomposed_id'] = d[names_decomposed].sum(axis=1).astype(str) + d['GENDER'] + d['birthdate'] 

# identifier 4 (first 3 letters of names - "Paso 5 identifier" + BIRTHDATE)
d['partialnames_id'] = d['FIRST_NAME_backup_letter1'] + d['FIRST_NAME_backup_letter2'] + d['FIRST_NAME_backup_letter3'] \
                     + d['LAST_NAME_backup_letter1'] + d['LAST_NAME_backup_letter2'] + d['LAST_NAME_backup_letter3'] \
                     + d['GENDER'] + d['birthdate']

# identifier 4b (first 3 letters of names - "Paso 5 identifier" + MOBILE_NUMBER)
d['partialnames_mobile_id'] = d['FIRST_NAME_backup_letter1'] + d['FIRST_NAME_backup_letter2'] + d['FIRST_NAME_backup_letter3'] \
                     + d['LAST_NAME_backup_letter1'] + d['LAST_NAME_backup_letter2'] + d['LAST_NAME_backup_letter3'] \
                     + d['GENDER'] + d['MOBILE_NUMBER']

# identifier 4c (first 3 letters of names - "Paso 5 identifier" + BIRTHYEAR + BIRTHMONTH)
d['partialnames_birthyearmonth_id'] = d['FIRST_NAME_backup_letter1'] + d['FIRST_NAME_backup_letter2'] + d['FIRST_NAME_backup_letter3'] \
                     + d['LAST_NAME_backup_letter1'] + d['LAST_NAME_backup_letter2'] + d['LAST_NAME_backup_letter3'] \
                     + d['GENDER'] + d['birthyear_str'] + d['birthmonth_str']

# identifier 4d (first 3 letters of names - "Paso 5 identifier" + BIRTHYEAR + BIRTHDAY)
d['partialnames_birthyearday_id'] = d['FIRST_NAME_backup_letter1'] + d['FIRST_NAME_backup_letter2'] + d['FIRST_NAME_backup_letter3'] \
                     + d['LAST_NAME_backup_letter1'] + d['LAST_NAME_backup_letter2'] + d['LAST_NAME_backup_letter3'] \
                     + d['GENDER'] + d['birthyear_str'] + d['birthday_str']

# identifier 4e (first 3 letters of names - "Paso 5 identifier" + BIRTHMONTH + BIRTHDAY)
d['partialnames_birthmonthday_id'] = d['FIRST_NAME_backup_letter1'] + d['FIRST_NAME_backup_letter2'] + d['FIRST_NAME_backup_letter3'] \
                     + d['LAST_NAME_backup_letter1'] + d['LAST_NAME_backup_letter2'] + d['LAST_NAME_backup_letter3'] \
                     + d['GENDER'] + d['birthmonth_str'] + d['birthday_str']


# identifier 5 (first 3 letters of separate names)
d['partialnames_decomposed_id'] = d[letter1].sum(axis=1).astype(str) + d[letter2].sum(axis=1).astype(str) + d[letter3].sum(axis=1).astype(str) + d['GENDER'] + d['birthdate']

# identifier 6 (Soundex - Non-decomposed names + BIRTHDATE)
d['soundex_id'] = d['FIRST_NAME_backup_soundex']+ d['MIDDLE_NAME_backup_soundex'] + d['LAST_NAME_backup_soundex'] \
                + d['GENDER'] + d['birthdate']

# identifier 6b (Soundex - Non-decomposed names + MOBILE_NUMBER)
d['soundex_mobile_id'] = d['FIRST_NAME_backup_soundex']+ d['MIDDLE_NAME_backup_soundex'] + d['LAST_NAME_backup_soundex'] \
                + d['GENDER'] + d['MOBILE_NUMBER']

# identifier 6c (Soundex - Non-decomposed names + BIRTHYEAR + BIRTHMONTH)
d['soundex_birthyearmonth_id'] = d['FIRST_NAME_backup_soundex']+ d['MIDDLE_NAME_backup_soundex'] + d['LAST_NAME_backup_soundex'] \
                + d['GENDER'] + d['birthyear_str'] + d['birthmonth_str']

# identifier 6d (Soundex - Non-decomposed names + BIRTHYEAR + BIRTHDAY)
d['soundex_birthyearday_id'] = d['FIRST_NAME_backup_soundex']+ d['MIDDLE_NAME_backup_soundex'] + d['LAST_NAME_backup_soundex'] \
                + d['GENDER'] + d['birthyear_str'] + d['birthday_str']

# identifier 6e (Soundex - Non-decomposed names + BIRTHMONTH + BIRTHDAY)
d['soundex_birthmonthday_id'] = d['FIRST_NAME_backup_soundex']+ d['MIDDLE_NAME_backup_soundex'] + d['LAST_NAME_backup_soundex'] \
                + d['GENDER'] + d['birthmonth_str'] + d['birthday_str']


# identifier 7 (Soundex - Decomposed names)
d['soundex_decomposed_id'] = d[soundex].sum(axis=1).astype(str) + d['GENDER'] + d['birthdate']

# identifier 8 (Metaphone - Non-decomposed names + BIRTHDATE)
d['metaphone_id'] = d['FIRST_NAME_backup_metaphone'] + d['MIDDLE_NAME_backup_metaphone'] + d['LAST_NAME_backup_metaphone'] \
                  + d['GENDER'] + d['birthdate']        

# identifier 8b (Metaphone - Non-decomposed names + MOBILE_NUMBER)
d['metaphone_mobile_id'] = d['FIRST_NAME_backup_metaphone'] + d['MIDDLE_NAME_backup_metaphone'] + d['LAST_NAME_backup_metaphone'] \
                  + d['GENDER'] + d['MOBILE_NUMBER']        

# identifier 8c (Metaphone - Non-decomposed names + BIRTHYEAR + BIRTHMONTH)
d['metaphone_birthyearmonth_id'] = d['FIRST_NAME_backup_metaphone'] + d['MIDDLE_NAME_backup_metaphone'] + d['LAST_NAME_backup_metaphone'] \
                  + d['GENDER'] + d['birthyear_str'] + d['birthmonth_str']     

# identifier 8d (Metaphone - Non-decomposed names + BIRTHYEAR + BIRTHDAY)
d['metaphone_birthyearday_id'] = d['FIRST_NAME_backup_metaphone'] + d['MIDDLE_NAME_backup_metaphone'] + d['LAST_NAME_backup_metaphone'] \
                  + d['GENDER'] + d['birthyear_str'] + d['birthday_str']     

# identifier 8e (Metaphone - Non-decomposed names + BIRTHMONTH + BIRTHDAY)
d['metaphone_birthmonthday_id'] = d['FIRST_NAME_backup_metaphone'] + d['MIDDLE_NAME_backup_metaphone'] + d['LAST_NAME_backup_metaphone'] \
                  + d['GENDER'] + d['birthmonth_str'] + d['birthday_str']     


# identifier 9 (Metaphone - Decomposed names)
d['metaphone_decomposed_id'] = d[metaphone].sum(axis=1).astype(str) + d['GENDER'] + d['birthdate']


In [41]:
# list of generated identifiers
id_list = ['raw_wholename_id',
           'clean_wholename_id','clean_wholename_mobile_id', 'clean_wholename_birthyearmonth_id', 'clean_wholename_birthyearday_id',
           'clean_wholename_birthmonthday_id','clean_wholename_decomposed_id',
           'partialnames_id','partialnames_mobile_id','partialnames_birthyearmonth_id','partialnames_birthyearday_id',
           'partialnames_birthmonthday_id','partialnames_decomposed_id',
           'soundex_id','soundex_mobile_id','soundex_birthyearmonth_id','soundex_birthyearday_id','soundex_birthmonthday_id',
           'soundex_decomposed_id',
           'metaphone_id','metaphone_mobile_id','metaphone_birthyearmonth_id','metaphone_birthyearday_id','metaphone_birthmonthday_id',
           'metaphone_decomposed_id']

#### Examine the duplicates

In [44]:
def duplicationanalyzer(data,id_list):
    ''' Assess the number and percentage of duplicates for each variable'''
    for var in id_list:
        # print the number of duplicates
        duplicates=data[data[var].duplicated(keep=False)]
        print(var)
        print('Number of duplicates: ',len(duplicates))

        # print the percentage of duplicates
        percentage = len(duplicates)/len(data)
        print('Percentage of duplicates: ',percentage)

In [45]:
duplicationanalyzer(d,id_list)

In [46]:
# generate dataframes with duplicates
dup1 = d[d['raw_wholename_id'].duplicated(keep=False)].sort_values(by=['raw_wholename_id'])
dup2 = d[d['clean_wholename_id'].duplicated(keep=False)].sort_values(by=['clean_wholename_id'])
dup3 = d[d['clean_wholename_mobile_id'].duplicated(keep=False)].sort_values(by=['clean_wholename_mobile_id'])
dup4 = d[d['clean_wholename_birthyearmonth_id'].duplicated(keep=False)].sort_values(by=['clean_wholename_birthyearmonth_id'])
dup5 = d[d['clean_wholename_birthyearday_id'].duplicated(keep=False)].sort_values(by=['clean_wholename_birthyearday_id'])
dup6 = d[d['clean_wholename_birthmonthday_id'].duplicated(keep=False)].sort_values(by=['clean_wholename_birthmonthday_id'])
dup7 = d[d['clean_wholename_decomposed_id'].duplicated(keep=False)].sort_values(by=['clean_wholename_decomposed_id'])

dup8 = d[d['partialnames_id'].duplicated(keep=False)].sort_values(by=['partialnames_id'])
dup9 = d[d['partialnames_mobile_id'].duplicated(keep=False)].sort_values(by=['partialnames_mobile_id'])
dup10 = d[d['partialnames_birthyearmonth_id'].duplicated(keep=False)].sort_values(by=['partialnames_birthyearmonth_id'])
dup11 = d[d['partialnames_birthyearday_id'].duplicated(keep=False)].sort_values(by=['partialnames_birthyearday_id'])
dup12 = d[d['partialnames_birthmonthday_id'].duplicated(keep=False)].sort_values(by=['partialnames_birthmonthday_id'])
dup13 = d[d['partialnames_decomposed_id'].duplicated(keep=False)].sort_values(by=['partialnames_decomposed_id'])

dup14 = d[d['soundex_id'].duplicated(keep=False)].sort_values(by=['soundex_id'])
dup15 = d[d['soundex_mobile_id'].duplicated(keep=False)].sort_values(by=['soundex_mobile_id'])
dup16 = d[d['soundex_birthyearmonth_id'].duplicated(keep=False)].sort_values(by=['soundex_birthyearmonth_id'])
dup17 = d[d['soundex_birthyearday_id'].duplicated(keep=False)].sort_values(by=['soundex_birthyearday_id'])
dup18 = d[d['soundex_birthmonthday_id'].duplicated(keep=False)].sort_values(by=['soundex_birthmonthday_id'])
dup19 = d[d['soundex_decomposed_id'].duplicated(keep=False)].sort_values(by=['soundex_decomposed_id'])

dup20 = d[d['metaphone_id'].duplicated(keep=False)].sort_values(by=['metaphone_id'])
dup21 = d[d['metaphone_mobile_id'].duplicated(keep=False)].sort_values(by=['metaphone_mobile_id'])
dup22 = d[d['metaphone_birthyearmonth_id'].duplicated(keep=False)].sort_values(by=['metaphone_birthyearmonth_id'])
dup23 = d[d['metaphone_birthyearday_id'].duplicated(keep=False)].sort_values(by=['metaphone_birthyearday_id'])
dup24 = d[d['metaphone_birthmonthday_id'].duplicated(keep=False)].sort_values(by=['metaphone_birthmonthday_id'])
dup25 = d[d['metaphone_decomposed_id'].duplicated(keep=False)].sort_values(by=['metaphone_decomposed_id'])

In [47]:
# list of duplicates
dup_list = [dup1, dup2, dup3, dup4, dup5, dup6, dup7, dup8, dup9, dup10, dup11, dup12, dup13, dup14, dup15, dup16, dup17, dup18,
           dup19, dup20, dup21, dup22, dup23, dup24, dup25]

#### Analysis of duplicates

In [48]:
dups =[] # list of total duplicates in each duplicates set
dup_pct =[] # list of percentage of duplicates from each duplicates set
for i in range(0,len(dup_list)):
    dups.append(len(dup_list[i]))
    dup_pct.append(len(dup_list[i])/len(d))
print(dups)
print(dup_pct)

Examine `STATUS` of duplicates

In [50]:
for dup in dup_list:
    print(dup['STATUS'].value_counts(dropna=False))

Examine outputs

In [51]:
dup1[['FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','REGION','MOBILE_NUMBER','raw_wholename_id']].head(10)

In [52]:
dup2[['FIRST_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','REGION','MOBILE_NUMBER','clean_wholename_id']].head(10)

In [53]:
dup3[['FIRST_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','REGION','MOBILE_NUMBER','clean_wholename_mobile_id']].head(10)

In [54]:
dup4[['FIRST_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','REGION','MOBILE_NUMBER','clean_wholename_birthyearmonth_id']].head(10)

In [55]:
dup5[['FIRST_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','REGION','MOBILE_NUMBER','clean_wholename_birthyearday_id']].head(10)

In [56]:
dup6[['FIRST_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','REGION','MOBILE_NUMBER','clean_wholename_birthmonthday_id']].head(10)

In [57]:
dup7[['FIRST_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','REGION','MOBILE_NUMBER','clean_wholename_decomposed_id']].head(10)

In [58]:
dup14[['FIRST_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','REGION','MOBILE_NUMBER','soundex_id']].head(10)

In [59]:
dup20[['FIRST_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','REGION','MOBILE_NUMBER','metaphone_id']].head(10)

In [60]:
#plot proportion of duplicates
x = ['Raw Whole Names','Clean Whole Names','Clean Whole Names - Mobile',
     'Clean Whole Names - Year + Month','Clean Whole Names - Year + Day','Clean Whole Names - Month + Day', 'Clean Whole Names - Decomposed',
     'Partial Names (Paso 5)','Partial Names - Mobile','Partial Names - Year + Month', 'Partial Names - Year + Day', 'Partial Names - Month + Day',
     'Partial Names - Decomposed',
     'Soundex','Soundex - Mobile','Soundex - Year + Month','Soundex - Year + Day','Soundex - Month + Day',
     'Soundex-Decomposed',
     'Metaphone','Metaphone - Mobile','Metaphone - Year + Month','Metaphone - Year + Day','Metaphone - Month + Day',
     'Metaphone-Decomposed']

fig, ax = plt.subplots(figsize=(15,7)) # Set figure size
ax = sns.barplot(x = x, y = dup_pct, palette = 'Blues', edgecolor = 'b')
ax.set_title("Duplicates as a percentage of total by Identifier", fontsize=18)
ax.set_xlabel("Identifier", fontsize=18)
ax.set_xticklabels(x, rotation=90, fontsize=14)
plt.show()

In [61]:
#plot total number of duplicates

fig, ax = plt.subplots(figsize=(15,7)) # Set figure size
ax = sns.barplot(x = x, y = dups, palette = 'Purples', edgecolor = 'b')
ax.set_title("Duplicates by Identifier", fontsize=18)
ax.set_xlabel("Identifier", fontsize=18)
ax.set_xticklabels(x, rotation=90, fontsize=14)
plt.show()

In [62]:
# plot comparing duplicates considered Active
active_dups =[]
for dup in dup_list:
    active_dups.append(len(dup[dup['STATUS']=='ACTIVE']))


fig, ax = plt.subplots(figsize=(15,7)) # Set figure size
ax = sns.barplot(x = x, y = active_dups, palette = 'Greens', edgecolor = 'b')
ax.set_title("Active Status Duplicates by Identifier", fontsize=18)
ax.set_xlabel("Identifier", fontsize=18)
ax.set_xticklabels(x, rotation=90, fontsize=14)
plt.show()

In [63]:
# plot comparing duplicates considered Not Active
inactive_dups =[]
for dup in dup_list:
    inactive_dups.append(len(dup[dup['STATUS']!='ACTIVE']))


fig, ax = plt.subplots(figsize=(15,7)) # Set figure size
ax = sns.barplot(x = x, y = inactive_dups, palette = 'Reds', edgecolor = 'b')
ax.set_title("Not Active Status Duplicates by Identifier", fontsize=18)
ax.set_xlabel("Identifier", fontsize=18)
ax.set_xticklabels(x, rotation=90, fontsize=14)
plt.show()

In [65]:
# outer join Clean Whole Names, Paso 5 ID, Soundex, Metaphone
dup1_results = dup1[['ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','STATUS','raw_wholename_id']]
dup2_results = dup2[['ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','STATUS','clean_wholename_id']]
dup8_results = dup8[['ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','STATUS','partialnames_id']]
dup14_results = dup14[['ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','STATUS','soundex_id']]
dup20_results = dup20[['ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','STATUS','metaphone_id']]

# mobile number results
dup3_results = dup3[['ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','STATUS','clean_wholename_mobile_id']]
dup9_results = dup9[['ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','STATUS','partialnames_mobile_id']]
dup15_results = dup15[['ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','STATUS','soundex_mobile_id']]
dup21_results = dup21[['ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','STATUS','metaphone_mobile_id']]

results_list = [dup1_results,dup2_results, dup8_results, dup14_results, dup20_results]

result1 = reduce(lambda left,right: pd.merge(left,right,on=['ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','STATUS'],how='outer'), results_list)
print(len(result1))

In [66]:
counts = [len(dup1_results),len(dup2_results),len(dup20_results),len(dup14_results),len(dup8_results)]
counts_pct = [len(dup1_results)/len(d),len(dup2_results)/len(d),
              len(dup20_results)/len(d),len(dup14_results)/len(d),len(dup8_results)/len(d)]
counts_pct=np.multiply(counts_pct,100)
count_labels = ['Raw Whole Names + Full Birthdate','Clean Whole Names + Full Birthdate',
                'Metaphone + Full Birthdate','Soundex + Full Birthdate','Partial Names (Paso 5) + Full Birthdate']
dup_df = pd.DataFrame(list(zip(count_labels,counts,counts_pct)),columns =['Labels','Counts','Counts Percentage'])
#index=list(reversed(range(8)))
dup_df

In [67]:
# plot counts
ax = dup_df.pivot(columns='Labels',values='Counts').plot(kind='barh',stacked=True,
                                                           colormap='Blues',rot=90,figsize=(15,7)) 
ax.set_title("Duplicates Identified by Biographic Methods in SCOPE for all Colombia", fontsize=16)
ax.legend(loc='center left', bbox_to_anchor=(1.05, 0.9), ncol=1)
ax.set_xlabel("Counts", fontsize=14)
ax.set_ylabel("Method", fontsize=14)
ax.set_yticklabels(dup_df.Labels, rotation=0, fontsize=14)

In [68]:
# plot percentage
ax = dup_df.pivot(columns='Labels',values='Counts Percentage').plot(kind='barh',stacked=True,
                                                           colormap='Reds',rot=90,figsize=(15,7)) 
ax.set_title("Duplicates Identified by Various Methods (in Percent of Total)", fontsize=16)
ax.legend(loc='center left', bbox_to_anchor=(1.05, 0.9), ncol=1)
ax.set_xlabel("Percent (%)", fontsize=14)
ax.set_ylabel("Method", fontsize=14)
ax.set_yticklabels(dup_df.Labels, rotation=0, fontsize=14)

In [69]:
catvardistribution(result1, 'STATUS', x=10,y=5,rot=60)

In [70]:
# outer join Clean Whole Names-Decomposed, Partial letters, Soundex-Decomposed, Metaphone-Decomposed
dup7_results = dup7[['ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','STATUS','clean_wholename_decomposed_id']]
dup13_results = dup13[['ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','STATUS','partialnames_decomposed_id']]
dup19_results = dup19[['ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','STATUS','soundex_decomposed_id']]
dup25_results = dup25[['ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','STATUS','metaphone_decomposed_id']]

results_list = [dup7_results, dup13_results, dup19_results, dup25_results]

result2 = reduce(lambda left,right: pd.merge(left,right,on=['ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','STATUS'],how='outer'), results_list)
print(len(result2))

In [71]:
catvardistribution(result2, 'STATUS', x=10,y=5,rot=60)

In [72]:
# compare clean whole names, soundex and metaphone identifiers
results_list = [dup2_results, dup3_results, dup14_results, dup15_results, dup20_results, dup21_results]

result3 = reduce(lambda left,right: pd.merge(left,right,on=['ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','GENDER','DATE_OF_BIRTH','STATUS'],how='outer'), results_list)
print(len(result3))

In [73]:
#high confidence duplicates
# inner join original data, and list of duplicates applying Clean Whole Names, Soundex and Metaphone methods
result3_copy = result3[['ID','clean_wholename_id','clean_wholename_mobile_id','soundex_id','soundex_mobile_id','metaphone_id','metaphone_mobile_id']]
results_list = [d_original,result3_copy]

result_highconfidence_scope = reduce(lambda left,right: pd.merge(left,right,on=['ID'],how='inner'), results_list)
result_highconfidence_scope = result_highconfidence_scope.sort_values(by=['soundex_id'])

# drop complete row duplicates
result_highconfidence_scope = result_highconfidence_scope.drop_duplicates(keep='first')
print(len(result_highconfidence_scope))

In [74]:
# STATUS distribution of high confidence duplicates
catvardistribution(result_highconfidence_scope, 'STATUS',x=7,y=5,rot=60)

In [75]:
# distribution of high confidence duplicates across regions
catvardistribution(data=result_highconfidence_scope,var='REGION',x=15,y=7,rot=30)

In [78]:
# drop unnecessary columns - retain REGION column
dup4_copy = dup4.drop(list(dup4.columns[75:92]),axis=1)
dup4_copy = dup4_copy.drop(list(dup4_copy.columns[76:]),axis=1)
dup6_copy = dup6.drop(list(dup6.columns[75:92]),axis=1)
dup6_copy = dup6_copy.drop(list(dup6_copy.columns[76:]),axis=1)
dup8_copy = dup8.drop(list(dup8.columns[75:92]),axis=1)
dup8_copy = dup8_copy.drop(list(dup8_copy.columns[76:]),axis=1)
result_highconfidence_scope_copy = result_highconfidence_scope.drop(list(result_highconfidence_scope.columns[75:92]),axis=1)
result_highconfidence_scope_copy = result_highconfidence_scope_copy.drop(list(result_highconfidence_scope_copy.columns[76:]),axis=1)

In [79]:
# export for review of potential false positives
dup4_copy.to_excel(f"{directory_output}/partialnames_duplicates.xlsx", index=False)
dup6_copy.to_excel(f"{directory_output}/soundex_duplicates.xlsx", index=False)
dup8_copy.to_excel(f"{directory_output}/metaphone_duplicates.xlsx", index=False)
result_highconfidence_scope_copy.to_excel(f"{directory_output}/duplicates_high_confidence_{date}.xlsx", index=False)

In [82]:
# remove high confidence duplicates from original dataset
duplicates_to_remove = result_highconfidence_scope

# set ID as the index
duplicates_to_remove = duplicates_to_remove.set_index('ID')
d_original = d_original.set_index('ID')

# data with partial names and high confidence duplicates from Arauca removed 
d_deduplicado = d_original.drop(duplicates_to_remove.index)

# reset index
#dup4_copy = dup4_copy.reset_index()
d_original = d_original.reset_index()
result_highconfidence_scope_copy = result_highconfidence_scope_copy.reset_index()
d_deduplicado = d_deduplicado.reset_index()

# IDs from exclude list removed
print(f'Total size of dataset after deduplication: {len(d_deduplicado)}')
print(len(d_original)-len(d_deduplicado))


In [83]:
# only heads of household
d_deduplicado_onlyheads = d_deduplicado[d_deduplicado.HOUSEHOLD_ROLE=='HEAD']
len(d_deduplicado_onlyheads)

In [84]:
# export the deduplicated dataset
d_deduplicado.to_excel(f"{directory_output}/no_duplicados_{date}.xlsx", index=False)
d_deduplicado_onlyheads.to_excel(f"{directory_output}/no_duplicados_solo_cabezas_{date}.xlsx", index=False)